In [7]:
import sympy as sp
import numpy as np
from sympy.core.assumptions import assumptions
sp.init_printing(use_unicode=True)

In [3]:
## initial variables and symbols

# time t
t = sp.Symbol('t')
steerControlInput = sp.Symbol('u_s')
torqueControlInput = sp.Symbol('tau_d')

# vehicle COG point G
xG = sp.Function('x_G')(t)
yG = sp.Function('y_G')(t)

# orientation of car and steering angle
phi = sp.Function('phi')(t)
psi = sp.Function('psi')(t)

xVehicle = sp.Function('x_g')(t)
dxVehicle = xVehicle.diff(t)

yVehicle = sp.Function('y_g')(t)
dyVehicle = yVehicle.diff(t)

lengthRearToMid, lengthFrontToMid, lengthTotal, T, m, r, K, J = sp.symbols('b d D T m r K J', nonnegative=True)

drivingForceRear = torqueControlInput / r

states = sp.Matrix([xG, yG, phi, dxVehicle, psi])
controls = sp.Matrix([torqueControlInput, steerControlInput])

evalAtZero = [(xG, 0), (yG, 0), (phi, 0), 
              (dxVehicle, 0), (psi, 0), 
              (torqueControlInput, 0), 
              (steerControlInput, 0)]

In [4]:
bTanPsiOverD = lengthRearToMid * sp.tan(psi) / lengthTotal
mbSqPlusJ = (m * lengthRearToMid**2) + J
accVehicleFCommonDenominator = sp.cos(psi)**2 * ((m*lengthTotal**2) + (mbSqPlusJ * sp.tan(psi)**2))

steerDriftTerm = - (1/T)*psi
steerControlTerm = (K / T)*steerControlInput


dxG = (sp.cos(phi) - (bTanPsiOverD * sp.sin(phi))) * dxVehicle
dyG = (sp.sin(phi) + (bTanPsiOverD * sp.cos(phi))) * dxVehicle
dphi = dxVehicle * sp.tan(psi) / lengthTotal
dpsi = steerControlTerm + steerDriftTerm

accVehicleFDriftTerm = (-1) * mbSqPlusJ * sp.tan(psi) * dpsi * dxVehicle / accVehicleFCommonDenominator
accVehicleFControlTerm = lengthTotal**2 * sp.cos(psi)**2 * drivingForceRear / accVehicleFCommonDenominator
ddxVehicle = accVehicleFDriftTerm + accVehicleFControlTerm

In [6]:
nonLinearVector = sp.Matrix([dxG, dyG, dphi, ddxVehicle, dpsi])

# \dot x = g_0(x) + g_1(x) tau_d + g_2(x) u_s
driftVector = sp.Matrix([dxG, dyG, dphi, accVehicleFDriftTerm, steerDriftTerm])
inputVectorTorque = sp.Matrix([0, 0, 0, accVehicleFControlTerm, 0])
inputVectorSteer = sp.Matrix([0, 0, 0, 0, steerControlTerm])

In [66]:
# this is a point in the state-space where the functions all go to 0
currentState = [(xG, 5), (yG, 3), (phi, np.pi/2), (dxVehicle, 0), (psi, np.pi/10), (torqueControlInput, 0), (steerControlInput, np.pi/10)]

symbolsEval = [(lengthTotal, 0.3), (lengthRearToMid, 0.3/2), (K, 1), (T, 0.01), (m, 0.5), (r, 0.05), (J, 0.03)]
nonLinearEvalAtPoint = nonLinearVector.subs(currentState)

display(nonLinearEvalAtPoint.subs(symbolsEval))

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

In [67]:
driftPartDeriv = driftVector.jacobian(states)
display(driftPartDeriv)

⎡                                                                             
⎢                                                                             
⎢      ⎛             b⋅cos(φ(t))⋅tan(ψ(t))⎞ d                           b⋅sin(
⎢0  0  ⎜-sin(φ(t)) - ─────────────────────⎟⋅──(x_g(t))      cos(φ(t)) - ──────
⎢      ⎝                       D          ⎠ dt                                
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢      ⎛            b⋅sin(φ(t))⋅tan(ψ(t))⎞ d                            b⋅cos(
⎢0  0  ⎜cos(φ(t)) - ─────────────────────⎟⋅──(x_g(t))       sin(φ(t)) + ──────
⎢      ⎝                      D          ⎠ dt                                 
⎢                                                                             
⎢                                                   

In [70]:
symbolsEval = [(lengthTotal, 0.3), (lengthRearToMid, 0.3/2), (K, 1), (T, 0.01), (m, 0.5), (r, 0.05), (J, 0.03)]

stateTransition = driftPartDeriv.subs(currentState)
display(stateTransition.subs(symbolsEval))

⎡0  0  0  -0.162459848116453    0   ⎤
⎢                                   ⎥
⎢0  0  0         1.0            0   ⎥
⎢                                   ⎥
⎢0  0  0   1.08306565410969     0   ⎥
⎢                                   ⎥
⎢0  0  0          0             0   ⎥
⎢                                   ⎥
⎣0  0  0          0           -100.0⎦

In [71]:
torquePartDeriv = inputVectorTorque.jacobian([controls[0]])
controlMatrixTorque = torquePartDeriv.subs(currentState)
display(torquePartDeriv, controlMatrixTorque)

⎡               0                ⎤
⎢                                ⎥
⎢               0                ⎥
⎢                                ⎥
⎢               0                ⎥
⎢                                ⎥
⎢                2               ⎥
⎢               D                ⎥
⎢────────────────────────────────⎥
⎢  ⎛ 2     ⎛     2  ⎞    2      ⎞⎥
⎢r⋅⎝D ⋅m + ⎝J + b ⋅m⎠⋅tan (ψ(t))⎠⎥
⎢                                ⎥
⎣               0                ⎦

⎡                           0                           ⎤
⎢                                                       ⎥
⎢                           0                           ⎥
⎢                                                       ⎥
⎢                           0                           ⎥
⎢                                                       ⎥
⎢                            2                          ⎥
⎢                           D                           ⎥
⎢───────────────────────────────────────────────────────⎥
⎢  ⎛ 2                                              2  ⎞⎥
⎢r⋅⎝D ⋅m + 0.105572809000084⋅J + 0.105572809000084⋅b ⋅m⎠⎥
⎢                                                       ⎥
⎣                           0                           ⎦

In [21]:
steerPartDeriv = inputVectorSteer.jacobian([controls[1]])
controlMatrixSteer = steerPartDeriv.subs(evalAtZero)
display(steerPartDeriv, controlMatrixSteer)

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢K⎥
⎢─⎥
⎣T⎦

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢K⎥
⎢─⎥
⎣T⎦

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢K⎥
⎢─⎥
⎣T⎦

In [18]:
controlMatrix = sp.Matrix([controlMatrixTorque.T, controlMatrixSteer.T]).T
controlMatrix

⎡ 0   0⎤
⎢      ⎥
⎢ 0   0⎥
⎢      ⎥
⎢ 0   0⎥
⎢      ⎥
⎢ 1    ⎥
⎢───  0⎥
⎢m⋅r   ⎥
⎢      ⎥
⎢     K⎥
⎢ 0   ─⎥
⎣     T⎦

In [44]:
smallAngles = 15 * np.pi / 180
x2 = np.linspace(-smallAngles, smallAngles, 50)
y = 2*np.ones(x2.shape)
x1 = np.arctan2(y, np.tan(x2))
xgdot = np.random.rand(50)

In [46]:
for a, b, c in zip(x1, x2, xgdot):
    nonLinearEval = nonLinearVector.subs([(xG, 5), (yG, 3), (phi, a), 
                                            (dxVehicle, c), (psi, b), 
                                            (torqueControlInput, 0), 
                                            (steerControlInput, 0), (lengthTotal, 0.3), (lengthRearToMid, 0.3/2)])
    print(nonLinearEval)

Matrix([[2.08166817117217e-17], [0.253106155143397], [-0.224063364667935], [-0.018861367561015*(-J - 0.0225*m)/(T*(0.0717967697244908*J + 0.091615427318801*m))], [0.261799387799149/T]])
Matrix([[5.55111512312578e-17], [0.811571212995423], [-0.688331652555160], [-0.0552672595698684*(-J - 0.0225*m)/(T*(0.0658069073528121*J + 0.0914806554154383*m))], [0.251113698501225/T]])
Matrix([[-2.08166817117217e-17], [0.287901905744978], [-0.233535270098127], [-0.0178570255630148*(-J - 0.0225*m)/(T*(0.0601085207886637*J + 0.0913524417177449*m))], [0.2404280092033/T]])
Matrix([[-5.20417042793042e-18], [0.0922882361637428], [-0.0714584145983052], [-0.00519448978080871*(-J - 0.0225*m)/(T*(0.0546959121073446*J + 0.0912306580224152*m))], [0.229742319905376/T]])
Matrix([[-3.03576608295941e-18], [0.0307174937275378], [-0.0226554298388665], [-0.00156263940115477*(-J - 0.0225*m)/(T*(0.0495637103577508*J + 0.0911151834830494*m))], [0.219056630607452/T]])
Matrix([[2.77555756156289e-17], [0.843527029716700], [-

In [31]:
# this is a point in the state-space where the functions all go to 0
nonLinearEvalAtPoint = nonLinearVector.subs([(xG, 5), (yG, 3), (phi, np.pi/2), 
                                            (dxVehicle, 0.01), (psi, np.pi/6), 
                                            (torqueControlInput, 0), 
                                            (steerControlInput, 0), (lengthTotal, 0.3), (lengthRearToMid, 0.3/2)])
print(nonLinearEvalAtPoint)

Matrix([[-0.00288675134594813], [0.0100000000000000], [0.0192450089729875], [-0.00403066525385382*(-J - 0.0225*m)/(T*(0.333333333333333*J + 0.0975*m))], [-0.523598775598299/T]])
